# convert the trajtory of sz GPS data to trips format for fitting the kde algorithm

In [2]:
import pickle

with open('datasets/bus_long50_traj_wgs84_coor/0_24_gps.pkl', 'rb') as f:
    data = pickle.load(f)

data

,traj,id,time,lon,lat,angle,speed,device,dspeed,mileage
0,0,BS01983D,2019-10-09 07:03:56+00:00,113.911102,22.658583,256.0,15.0,1638196,0.0,48147.0
1,0,BS01983D,2019-10-09 07:04:10+00:00,113.910248,22.658617,279.0,12.0,1638196,0.0,48147.0
2,0,BS01983D,2019-10-09 07:04:14+00:00,113.910149,22.658617,274.0,0.0,1638196,0.0,48147.0
3,0,BS01983D,2019-10-09 07:04:28+00:00,113.909515,22.658617,271.0,28.0,1638196,0.0,48147.0
4,0,BS01983D,2019-10-09 07:04:29+00:00,113.909431,22.658634,272.0,27.0,1638196,0.0,48147.0
...,...,...,...,...,...,...,...,...,...,...
83259,2003,BS43022D,2019-10-09 21:47:40+00:00,113.911453,22.657784,159.0,14.0,1701105,0.0,53401.0
83260,2003,BS43022D,2019-10-09 21:47:45+00:00,113.911499,22.657700,155.0,0.0,1701105,0.0,53401.0
83261,2003,BS43022D,2019-10-09 21:47:59+00:00,113.911697,22.657484,141.0,16.0,1701105,0.0,53401.0
83262,2003,BS43022D,2019-10-09 21:48:02+00:00,113.911835,22.657316,143.0,20.0,1701105,0.0,53401.0


In [3]:
# split traj to trips, and convert the data time to timestamp

import pandas as pd
import os

# Load the pickle file
df = pd.read_pickle('datasets/bus_long50_traj_wgs84_coor/0_24_gps.pkl')

# Define the columns to keep
columns_to_keep = ['id', 'lat', 'lon', 'time']

# Get unique traj values
unique_trajs = df['traj'].unique()

# Save each unique traj to a separate text file
save_path = 'datasets/trips_0_24_new'
if not os.path.exists(save_path):
    os.makedirs(save_path)
for traj in unique_trajs:
    traj_df = df[df['traj'] == traj][columns_to_keep]
    traj_df['time'] = pd.to_datetime(traj_df['time']).apply(lambda x: x.timestamp())
    #如果一个traj中存在两条数据的time相同，那么就不保存这个traj
    # traj_df = traj_df.drop_duplicates(subset='time')
    if len(traj_df) != len(traj_df['time'].unique()):
        print(f'Trip {traj} has duplicated timestamps, skipping...')
        continue

    traj_df.to_csv(f'{save_path}/trip_{traj}.txt', header=False, index=False)
print('Done!')


Trip 1 has duplicated timestamps, skipping...
Trip 3 has duplicated timestamps, skipping...
Trip 4 has duplicated timestamps, skipping...
Trip 5 has duplicated timestamps, skipping...
Trip 10 has duplicated timestamps, skipping...
Trip 11 has duplicated timestamps, skipping...
Trip 14 has duplicated timestamps, skipping...
Trip 16 has duplicated timestamps, skipping...
Trip 18 has duplicated timestamps, skipping...
Trip 19 has duplicated timestamps, skipping...
Trip 20 has duplicated timestamps, skipping...
Trip 21 has duplicated timestamps, skipping...
Trip 22 has duplicated timestamps, skipping...
Trip 24 has duplicated timestamps, skipping...
Trip 25 has duplicated timestamps, skipping...
Trip 36 has duplicated timestamps, skipping...
Trip 37 has duplicated timestamps, skipping...
Trip 38 has duplicated timestamps, skipping...
Trip 39 has duplicated timestamps, skipping...
Trip 41 has duplicated timestamps, skipping...
Trip 43 has duplicated timestamps, skipping...
Trip 44 has dupli

In [4]:
print(len(os.listdir('datasets/trips_0_24_new')))

150
